In [7]:
import pandas as pd

first = [1, 2, {"a": "ab", "b": "bb"}]
second = [3, 4, {"c": "cb", "d": "dd"}]
a = pd.DataFrame([first, second] )
print(a)

   0  1                       2
0  1  2  {'a': 'ab', 'b': 'bb'}
1  3  4  {'c': 'cb', 'd': 'dd'}


In [2]:
from lxml import html
import requests
import pandas as pd
import time


__author__ = 'aituarov'


DB_LOADER_DIR = 'C:\\DEV\\Dias\\'


header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}

category_urls = ['https://shop.akpo.by/vytyazhki',
                'https://shop.akpo.by/varochnye-paneli',
                 'https://shop.akpo.by/duhovye-shkafy',
                 'https://shop.akpo.by/posudomoechnye-mashiny',
                 'https://shop.akpo.by/mikrovolnovye-pechi',
                 'https://shop.akpo.by/aksessuary'
                ]

def get_product_urls():
    main_url_products = set()
    
    for main_url in category_urls:
        main_response = requests.get(main_url, headers=header)
        main_page = html.fromstring(main_response.text)

        page_urls = {main_url}
        product_page_urls_el = main_page.xpath('//ul[@class="pagination"]/li/a')
        [page_urls.add(page_el.get('href')) for page_el in product_page_urls_el]


        for url in page_urls:
            print("Getting product urls from " + url)
            response = requests.get(url, headers=header)
            page = html.fromstring(response.text)
            product_div_els = page.xpath('//div[@id="content"]/div[@class="row"]')[0]
            products = product_div_els.xpath('//div[@class="product-thumb"]//div[@class="caption"]/h4/a')
            for product in products:
                main_url_products.add(product.get('href'))

        page_urls.clear()
            
    return main_url_products



def get_products_info(urls_set):
    data = pd.DataFrame()
    
    for url in urls_set:
        category = url.split('/')[-2]

        response = requests.get(url, headers=header)
        page = html.fromstring(response.text)

        name = page.xpath('//h1[@class="pr-name"]')[0].text

        price = page.xpath('//span[@class="autocalc-product-price"]')[0].text

        images_urls = []
        images_el = page.xpath('//div[@id="fix_image"]//a')
        for image_el in images_el:
            images_urls.append(image_el.get('href'))

        characters = {}
        char_rows = page.xpath('//div[@class="tab-content"]//tbody/tr')
        for row in char_rows:
            char_name = row.xpath('.//td')[0].text
            char_value = row.xpath('.//td')[1].text
            characters[char_name] = char_value

        data = data.append([[category, url, name, price, images_urls, characters]], ignore_index=True)
        print("Got data from " + url)
        
    data.columns = ['category','url', 'name', 'price', 'images','characters']
    print(data)
#     data.to_excel(DB_LOADER_DIR + 'new_products.xlsx', sheet_name = 'AKPO')
#     print("End.")
#     print(data)

    
    
def main():
    product_urls = get_product_urls()
    print("\nProduct urls collected")
    
    get_products_info(product_urls)


    
    
if __name__ == '__main__':
    main()


ModuleNotFoundError: No module named 'lxml'

In [175]:
from lxml import html
import requests
import pandas as pd
import time


__author__ = 'aituarov'


DB_LOADER_DIR = 'C:\\DEV\\Dias\\'

category_urls = ['https://ecostone73.ru/moiki',
                 'https://ecostone73.ru/smesiteli',
                 'https://ecostone73.ru/dozators'
                ]


def get_product_urls():
    main_url_products = set()
    
    for main_url in category_urls:
        main_response = requests.get(main_url)
        main_page = html.fromstring(main_response.text)
        
        products_urls = main_page.xpath('//section[@id="section-content"]//a')
        for a in products_urls:
            main_url_products.add(main_url + a.get('href'))
            print('Got data from ' + main_url + a.get('href'))

    return main_url_products


def get_products_info(urls_set):
    data = pd.DataFrame()
    
    for url in urls_set:
        category = url.split('/')[-3]

        url = url.replace('/' + category, '')
        response = requests.get(url)
        page = html.fromstring(response.text)

        name=''
        name_el = page.xpath('//p[@class="moto-text_system_5"]/span')
        for span in name_el:
            if span.text!= None:
                name+= span.text

        try:
            description = page.xpath('//p[@class="moto-text_218"]/strong')[0].text
        except IndexError:
            description = ''

        guarantee = page.xpath('//p[@class="moto-text_461"]//text()')[1]

        characters = {'Гарантия:': guarantee}
        characters_el = page.xpath('//p[@class="moto-text_461"]//text()')
        for ind in range(len(characters_el)):
            if characters_el[ind].endswith(':'):
                characters[characters_el[ind]] = characters_el[ind+1]
            elif characters_el[ind].startswith('•'):
                characters[len(characters)] = characters_el[ind]

        images_urls = []
        image_front = page.xpath('//section[@id="section-content"]//span[@class="moto-widget-image-link"]/img')[0].get('data-src')
        images_urls.append(url + image_front)
        image_scheme = page.xpath('//p[contains(.,"Схема:")]/../../../..//img')[0].get('src')
        images_urls.append(url + image_scheme)
    
        data = data.append([[category, url, name, None, images_urls, description, characters]], ignore_index=True)
        print("Got data from " + url)
        
    data.columns = ['category','url', 'name', 'price', 'images', 'description','characters']
    data.to_excel(DB_LOADER_DIR + 'products_2.xlsx', sheet_name = 'ECOSTONE')
    print("End.")



def main():
    product_urls = get_product_urls()
    print("\nProduct urls collected")
    
    get_products_info(product_urls)


    
    
if __name__ == '__main__':
    main()

Got data from https://ecostone73.ru/moiki/es-10/
Got data from https://ecostone73.ru/moiki/es-10/
Got data from https://ecostone73.ru/moiki/es-10/
Got data from https://ecostone73.ru/moiki/es-11/
Got data from https://ecostone73.ru/moiki/es-11/
Got data from https://ecostone73.ru/moiki/es-11/
Got data from https://ecostone73.ru/moiki/es-12/
Got data from https://ecostone73.ru/moiki/es-12/
Got data from https://ecostone73.ru/moiki/es-12/
Got data from https://ecostone73.ru/moiki/es-13/
Got data from https://ecostone73.ru/moiki/es-13/
Got data from https://ecostone73.ru/moiki/es-13/
Got data from https://ecostone73.ru/moiki/es-14/
Got data from https://ecostone73.ru/moiki/es-14/
Got data from https://ecostone73.ru/moiki/es-14/
Got data from https://ecostone73.ru/moiki/es-15/
Got data from https://ecostone73.ru/moiki/es-15/
Got data from https://ecostone73.ru/moiki/es-15/
Got data from https://ecostone73.ru/moiki/es-16/
Got data from https://ecostone73.ru/moiki/es-16/
Got data from https: